In [1]:
import sqlite3
import pandas as pd

In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

Creating DB Schema

In [25]:
con = sqlite3.connect("secure_shield.db")
c = con.cursor()

In [26]:
c.execute("PRAGMA foreign_keys = ON;")

In [ ]:

#c.execute("SELECT name FROM sqlite_master WHERE type='table';")
#tables = c.fetchall()

# Drop each table
#for table in tables:
 #   table_name = table[0]
  #  c.execute(f"DROP TABLE IF EXISTS {table_name}")

In [56]:
c.execute('''
          CREATE TABLE Employees (
          employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
          first_name TEXT NOT NULL,
          last_name TEXT NOT NULL,
          email TEXT UNIQUE NOT NULL,
          role CHECK(role IN ('Claims Adjuster', 'Manager')),
          password TEXT NOT NULL,
          conversation_id INTEGER DEFAULT 0)
          ''')

In [61]:
c.execute('''
          CREATE TABLE Clients (
          client_id INTEGER PRIMARY KEY AUTOINCREMENT,
          name TEXT NOT NULL,
          age INTEGER NOT NULL,
          gender TEXT NOT NULL,
          email TEXT UNIQUE NOT NULL,
          phone TEXT NOT NULL,
          policy_id INTEGER,
          FOREIGN KEY (policy_id) REFERENCES Policies(policy_id)
          )
          ''')

In [91]:
c.execute('''
          CREATE TABLE Claims (
          claim_id INTEGER PRIMARY KEY AUTOINCREMENT,
          user_id INTEGER NOT NULL,
          policy_id INTEGER NOT NULL,
          claim_type TEXT NOT NULL,
          status TEXT NOT NULL,
          submission_date DATE NOT NULL,
          resolution_date DATE,
          amount_claimed REAL NOT NULL,
          amount_approved REAL,
          approved_by INTEGER,
          FOREIGN KEY (user_id) REFERENCES Clients(client_id),
          FOREIGN KEY (policy_id) REFERENCES Policies(policy_id),
          FOREIGN KEY (approved_by) REFERENCES Employees(employee_id)
          );
          ''')

In [58]:
c.execute('''
          CREATE TABLE Policies (
          policy_id INTEGER PRIMARY KEY AUTOINCREMENT,
          user_id INTEGER NOT NULL,
          policy_type CHECK(policy_type IN ('Health', 'House', 'Car')),
          policy_level CHECK(policy_level IN ('Bronze', 'Silver', 'Gold')),
          start_date DATE NOT NULL,
          end_date DATE NOT NULL,
          pdf_path TEXT,
          FOREIGN KEY (user_id) REFERENCES Clients(client_id)
          )
         ''')

In [31]:
con.commit()
con.close()

Generating data

In [4]:
def get_answer(prompt, model='gpt-3.5-turbo', **kwargs):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        **kwargs)
    return response.choices[0].message.content

In [5]:
clients = get_answer(prompt=f"""
Generate a list of twenty made-up clients of a online platform along \
with their name, age, gender, email and phone.
Provide them in JSON format with the following keys:
name, age, gender, email and phone.
""", temperature=0.6)

In [6]:
clients = pd.read_json(clients)
final_clients = pd.json_normalize(clients['clients'])

/tmp/ipykernel_2084/797118326.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  clients = pd.read_json(clients)


In [7]:
final_clients.head()

,name,age,gender,email,phone
0,Alice Johnson,28,Female,alice.johnson@example.com,555-1234
1,John Smith,35,Male,john.smith@example.com,555-5678
2,Emily Davis,42,Female,emily.davis@example.com,555-9101
3,Michael Brown,50,Male,michael.brown@example.com,555-1122
4,Sophia Wilson,22,Female,sophia.wilson@example.com,555-3344


In [8]:
employees = get_answer(prompt=f"""
Generate a list of twenty made-up employees of a online platform along \
with their first and last name, email, password and role that can be 'Claims Adjuster' or 'Manager'.
Provide them in JSON format with the following keys:
first_name, last_name, email, role and password.
""", temperature=0.6)

In [9]:
employees = pd.read_json(employees)
final_employees = pd.json_normalize(employees['employees'])

/tmp/ipykernel_2084/1411261104.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  employees = pd.read_json(employees)


In [10]:
final_employees.head()

,first_name,last_name,email,password,role
0,Samantha,Smith,samantha.smith@example.com,P@ssw0rd,Manager
1,Michael,Johnson,michael.johnson@example.com,P@ssw0rd,Claims Adjuster
2,Emily,Davis,emily.davis@example.com,P@ssw0rd,Manager
3,Daniel,Martinez,daniel.martinez@example.com,P@ssw0rd,Claims Adjuster
4,Jennifer,Lee,jennifer.lee@example.com,P@ssw0rd,Manager


In [ ]:
policies = get_answer(prompt=f"""
Generate a list of twenty made-up policies of a online platform along with their policy type('Health' or 'House' or 'Car'), policy level('Bronze' or 'Silver' or 'Gold') the policy's start date and expiration date and pdf_path that if polity type is health than it must be 'Policies/HealthCare.pdf', if polity type is house than it must be 'Policies/HomeProtect.pdf'and if polity type is car than it must be 'Policies/AutoGuard.pdf'.
            
                      

Provide them in JSON format with the following keys:
policy_type, policy_level, start_date, end_date and pdf_path.
""", temperature=0.6)

In [14]:
policies = pd.read_json(policies)
final_policies = pd.json_normalize(policies['policies'])

/tmp/ipykernel_2084/2120300166.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  policies = pd.read_json(policies)


In [15]:
final_policies.head()

,policy_type,policy_level,start_date,end_date,pdf_path
0,Health,Bronze,2021-01-01,2021-12-31,Policies/HealthCare.pdf
1,House,Silver,2021-02-15,2022-02-14,Policies/HomeProtect.pdf
2,Car,Gold,2021-03-10,2022-03-09,Policies/AutoGuard.pdf
3,Health,Silver,2021-04-20,2022-04-19,Policies/HealthCare.pdf
4,House,Bronze,2021-05-05,2022-05-04,Policies/HomeProtect.pdf


In [22]:
claims = get_answer(prompt=f"""
Generate a list of twenty made-up claims of a online platform along \
claim type(for example: vehicle accident, theft, medical ...), current status of the claim (pending, approved, denied), the date when the claim was filed, the date when the claim was resolved (can be null), the total amount claimed by the user, the amount approved for reimbursement (can be null).
Provide them in JSON format with the following keys:
claim_type, status, submission_date, resolution_date, amount_claimed, amount_approved.
""", temperature=0.6)

In [23]:
claims = pd.read_json(claims)
final_claims = pd.json_normalize(claims['claims'])

/tmp/ipykernel_2084/4208098615.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  claims = pd.read_json(claims)


In [24]:
final_claims.head()

,claim_type,status,submission_date,resolution_date,amount_claimed,amount_approved
0,vehicle accident,approved,2021-05-10,2021-06-05,5000,4000.0
1,theft,denied,2021-04-15,None,3000,NaN
2,medical,pending,2021-06-20,None,10000,NaN
3,property damage,approved,2021-07-01,2021-07-20,7000,6000.0
4,lost luggage,denied,2021-03-25,None,2000,NaN


In [46]:
final_clients.to_csv('clients.csv', index=False)
final_claims.to_csv('claims.csv', index=False)
final_employees.to_csv('employees.csv', index=False)
final_policies.to_csv('policies.csv', index=False)

In [47]:
clients = pd.read_csv('clients.csv')
claims = pd.read_csv('claims.csv')
employees = pd.read_csv('employees.csv')
policies = pd.read_csv('policies.csv')
clients.head()

,name,age,gender,email,phone
0,Alice Johnson,28,Female,alice.johnson@example.com,555-1234
1,John Smith,35,Male,john.smith@example.com,555-5678
2,Emily Davis,42,Female,emily.davis@example.com,555-9101
3,Michael Brown,50,Male,michael.brown@example.com,555-1122
4,Sophia Wilson,22,Female,sophia.wilson@example.com,555-3344


Populating the DB

In [32]:
con = sqlite3.connect("secure_shield.db")
c = con.cursor()

In [62]:
for u in clients.name:
    row = clients[clients['name'] == u].iloc[0] 

    values = (
        row['name'],
        row['age'],
        row['gender'],
        row['email'],
        row['phone'])
    
    c.execute("INSERT INTO clients ( name, age, gender, email, phone) VALUES ( ?,?,?,?,?)", values)
con.commit()


In [66]:
for u in employees.first_name:
    row =  employees[ employees['first_name'] == u].iloc[0] 

    values = (
        row['first_name'],
        row['last_name'],
        row['email'],
        row['role'],
        row['password'])
    
    c.execute("INSERT INTO employees (first_name, last_name, email, role, password) VALUES ( ?,?,?,?,?)", values)
con.commit()

In [71]:
policies.append()

,policy_type,policy_level,start_date,end_date,pdf_path
0,Health,Bronze,2021-01-01,2021-12-31,Policies/HealthCare.pdf
1,House,Silver,2021-02-15,2022-02-14,Policies/HomeProtect.pdf
2,Car,Gold,2021-03-10,2022-03-09,Policies/AutoGuard.pdf
3,Health,Silver,2021-04-20,2022-04-19,Policies/HealthCare.pdf
4,House,Bronze,2021-05-05,2022-05-04,Policies/HomeProtect.pdf
5,Car,Silver,2021-06-30,2022-06-29,Policies/AutoGuard.pdf
6,Health,Gold,2021-07-10,2022-07-09,Policies/HealthCare.pdf
7,House,Gold,2021-08-25,2022-08-24,Policies/HomeProtect.pdf
8,Car,Bronze,2021-09-05,2022-09-04,Policies/AutoGuard.pdf
9,Health,Silver,2021-10-15,2022-10-14,Policies/HealthCare.pdf


In [72]:

# Retrieve client_id from the Clients table
c.execute("SELECT client_id FROM Clients;")
client_ids = [row[0] for row in c.fetchall()]  # Fetch all client_ids and store them in a list

policies['user_id'] = client_ids[:len(policies)]  # Assign random client_ids to the policies DataFrame


   policy_type policy_level  start_date    end_date                  pdf_path  \
0       Health       Bronze  2021-01-01  2021-12-31   Policies/HealthCare.pdf   
1        House       Silver  2021-02-15  2022-02-14  Policies/HomeProtect.pdf   
2          Car         Gold  2021-03-10  2022-03-09    Policies/AutoGuard.pdf   
3       Health       Silver  2021-04-20  2022-04-19   Policies/HealthCare.pdf   
4        House       Bronze  2021-05-05  2022-05-04  Policies/HomeProtect.pdf   
5          Car       Silver  2021-06-30  2022-06-29    Policies/AutoGuard.pdf   
6       Health         Gold  2021-07-10  2022-07-09   Policies/HealthCare.pdf   
7        House         Gold  2021-08-25  2022-08-24  Policies/HomeProtect.pdf   
8          Car       Bronze  2021-09-05  2022-09-04    Policies/AutoGuard.pdf   
9       Health       Silver  2021-10-15  2022-10-14   Policies/HealthCare.pdf   
10       House       Bronze  2021-11-20  2022-11-19  Policies/HomeProtect.pdf   
11         Car         Gold 

In [73]:
for u in policies.policy_type:
    row =  policies[ policies['policy_type'] == u].iloc[0] 

    values = (
        row['policy_type'],
        row['policy_level'],
        row['start_date'],
        row['end_date'],
        row['pdf_path'],
        row['user_id'])
    
    c.execute("INSERT INTO policies (policy_type, policy_level, start_date, end_date, pdf_path, user_id) VALUES ( ?,?,?,?,?,?)", values)
con.commit()

In [87]:
c.execute("SELECT client_id FROM Clients;")
client_ids = [row[0] for row in c.fetchall()]
claims['user_id'] = client_ids[:len(claims)]

c.execute("SELECT policy_id FROM Policies;")
policy_ids = [row[0] for row in c.fetchall()]
claims['policy_id'] = policy_ids[:len(claims)]

c.execute("SELECT employee_id FROM Employees;")
employee_ids = [row[0] for row in c.fetchall()]
claims['approved_by'] = employee_ids[:len(claims)]

In [94]:
for u in claims.status:
    row =  claims[claims['status'] == u].iloc[0] 

    values = (
        row['claim_type'],
        row['status'],
        row['submission_date'],
        row['resolution_date'],
        row['amount_claimed'],
        row['amount_approved'],
        row['user_id'],
        row['policy_id'],
        row['approved_by'])
    
    c.execute("INSERT INTO claims (claim_type, status, submission_date, resolution_date, amount_claimed, amount_approved, user_id, policy_id, approved_by) VALUES ( ?,?,?,?,?,?,?,?,?)", values)
con.commit()